In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append("..")
from src.dataset import SIModel
from src.utils import x_loglikelihood, contact_matrix
import seaborn as sns
from scipy.optimize import minimize_scalar, minimize
from scipy.stats import norm, multivariate_normal, probplot
import emcee

In [ ]:
sns.set_theme()
beta_true = .15
alpha = 0.1
gamma = 0.05
N = 100
T = 52
seed = 29

In [ ]:
F = np.arange(N) % 5
R = np.arange(N) % (N // 2)
fC = contact_matrix(F)
rC = contact_matrix(R)

In [ ]:
model = SIModel(alpha, gamma, beta_true, False, -3, 1, N, T, summarize=False, observed_seed=seed)
X_o = model.get_observed_data(29)[0].numpy()
I_o = X_o.sum(0)

In [ ]:
plt.plot(I_o / N, label="Infected")
plt.legend(loc="upper right")
plt.xlabel("Time Steps")
plt.ylabel("Proportion")
plt.ylim(0, 1.1)
plt.title("Susceptible-Infected Simulation with Migration")
plt.show()

# Homogeneous Transmission

In [ ]:
def f(logbeta):
    return - x_loglikelihood(logbeta, alpha, gamma, N, T, X_o, het=False)

res = minimize_scalar(f, options={"disp": True, "maxiter": 18})
res.x

In [ ]:
np.exp(res.x)

In [ ]:
def log_prob_homog(logbeta, X, alpha, gamma, N, T, het, prior_mu):
    prior = norm(prior_mu)
    lp = prior.logpdf(logbeta)
    return lp + x_loglikelihood(logbeta, alpha, gamma, N, T, X, het)

In [ ]:
nwalkers = 4
pos = res.x + 1e-3 * np.random.randn(nwalkers, 1)
prior_mu = -2
sampler = emcee.EnsembleSampler(
    nwalkers, 1, log_prob_homog, args = (X_o, alpha, gamma, N, T, False, prior_mu)
)
sampler.run_mcmc(pos, 2000, progress=True)

flat_samples = sampler.get_chain(discard=100, thin=50, flat=True)

In [ ]:
flat_samples.mean(), flat_samples.std()

In [ ]:
np.exp(flat_samples).mean()

In [ ]:
plt.hist(flat_samples)
plt.show()

In [ ]:
fig, ax = plt.subplots()
probplot(flat_samples[:, 0], dist="norm", plot=ax)
plt.show()

# Heterogeneous Transmission

In [ ]:
beta_true = np.array([.05, .02, .04, .06, .08, .1, .05])
alpha = 0.1
gamma = 0.05
heterogeneous = True
N = 300 # consider increasing
T = 52
K = 30
seed = 31
prior_mu = -3

In [ ]:
# X_o = simulator(alpha, beta_true, gamma, N, T, seed=31, het=True)
# I_o = X_o.sum(0).astype(int)
model = SIModel(alpha, gamma, beta_true, True, -3, 1, N, T, summarize=False, observed_seed=seed)
X_o = model.get_observed_data(seed)[0].numpy()
I_o = X_o.sum(0)
plt.plot(I_o / N, label="Infected")
plt.legend(loc="upper right")
plt.xlabel("Time Steps")
plt.ylabel("Proportion")
plt.ylim(0, 1.1)
plt.title("Susceptible-Infected Simulation with Migration")
plt.show()

In [ ]:
def g(logbeta, alpha, gamma, N, T, X_o, het=True):
    return - x_loglikelihood(logbeta, alpha, gamma, N, T, X_o, het=True)

res2 = minimize(
    g, x0 = np.log(beta_true), args = (alpha, gamma, N, T, X_o, True), tol=0.0001
)
res2.x

In [ ]:
np.exp(res2.x).round(3)

In [ ]:
# res2 = minimize(
#     utils.nll, x0 = beta_true, args = (alpha, gamma, N, T, X_o, True),
#     bounds = [(0.0, None) for _ in range(7)], tol=0.0001
# )
# res2.x

In [ ]:
def log_prob_het(logbeta, X, alpha, gamma, N, T, het, prior_mu):
    prior = multivariate_normal(np.full(7, prior_mu))
    lp = prior.logpdf(logbeta)
    return lp + x_loglikelihood(logbeta, alpha, gamma, N, T, X, het)

In [ ]:
nwalkers = 16
initial = np.full(7, -3)
pos = res2.x + 1e-3 * np.random.randn(nwalkers, 7)
sampler_het = emcee.EnsembleSampler(
    nwalkers, 7, log_prob_het, args = (X_o, alpha, gamma, N, T, True, prior_mu)
)
# up to 4,000
sampler_het.run_mcmc(pos, 2000, progress=True, skip_initial_state_check=True)
flat_samples_het = sampler_het.get_chain(discard=500, thin=100, flat=True)

In [ ]:
np.exp(flat_samples_het).mean(0)

In [ ]:
(flat_samples_het).std(0)

## Modified Prior

A more conservative prior.

In [ ]:
new_prior = np.array([-3, -4, -4, -4, -4, -4, -4])

In [ ]:
def log_prob_het2(logbeta, X, alpha, gamma, N, T, het):
    prior = multivariate_normal(new_prior)
    lp = prior.logpdf(logbeta)
    return lp + x_loglikelihood(logbeta, alpha, gamma, N, T, X, het)

In [ ]:
nwalkers = 16
initial = np.full(7, -3)
pos = res2.x + 1e-3 * np.random.randn(nwalkers, 7)
sampler_het2 = emcee.EnsembleSampler(
    nwalkers, 7, log_prob_het2, args = (X_o, alpha, gamma, N, T, True)
)
# up to 4,000
sampler_het2.run_mcmc(pos, 2000, progress=True, skip_initial_state_check=True)
flat_samples_het2 = sampler_het2.get_chain(discard=500, thin=100, flat=True)

In [ ]:
np.exp(flat_samples_het).mean(0)

In [ ]:
np.exp(flat_samples_het2).mean(0)

In [ ]:
flat_samples_het2.mean(0)

In [ ]:
flat_samples_het2.std(0)

In [ ]:
np.exp(res2.x).round(4)

## Correlation Patterns

In [ ]:
sns.pairplot(pd.DataFrame(flat_samples_het))
plt.show()

In [ ]:
sns.pairplot(pd.DataFrame(np.exp(flat_samples_het2)))
plt.show()